In [1]:
import bmll2 as b2
b2.get_file('modules/auxiliary_functions.py')

'auxiliary_functions.py'

In [3]:
# can do this when i have converted the notebooks to .py files
# import auxiliary_functions
import auxiliary_functions as af

import random
import math
import pandas as pd
import numpy as np
from pandas import StringDtype

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from matplotlib.ticker import LogFormatterSciNotation

In [ ]:
b2.get_file('test_data/homo_4_immediate(100).csv')
impact_data = pd.read_csv('homo_4_immediate(100).csv', parse_dates = ['Date', 'Start time', 'End time'])

GRT_AD = impact_data[impact_data['RIC'] == 'GRT'][['RIC', 'Date', '20 AD volatility', '20 AD volume']]
GFI_AD = impact_data[impact_data['RIC'] == 'GFI'][['RIC', 'Date', '20 AD volatility', '20 AD volume']]

In [25]:
#b2.get_file('top_40/GRT.csv')
#GFI = pd.read_csv('GRT.csv', parse_dates = ['DateTime', 'Date'])
#GFI = GFI.rename(columns = {'Ticker' : 'RIC'})
#print(GFI.Volume.sum(), GFI.shape[0])
#1 287 444 156.0,    54 194 RMH
#5 436 331 147.0, 2 254 246 GRT
#  685 548 799.0, 4 919 457 AGL
#6 464 855 862.0, 2 512 893 OMU
#2 433 105 568.0, 6 056 742 MTN
#1 310 645 090.0, 9 668 090 GFI 


5436331147.0 2254246


In [11]:
%%time
b2.get_file('top_100(Volume)/GFI.csv')
GFI = pd.read_csv('GFI.csv', parse_dates = ['DateTime', 'Date'])
GFI = GFI.rename(columns = {'Ticker' : 'RIC'})
GFI = GFI.sort_values(['DateTime', 'ExchangeSequenceNo'])

impact_GFI = pd.DataFrame(columns = ['RIC', 'Date', 'Start time', 'End time', 'daily volume', 'intraday volatility',
                                     'number child orders', 'volume traded', 'trade sign', 'impact(shortfall)',
                                     'impact(ave per trade)', 'impact(simple)', '20 AD volatility'])

impact_GFI = []
for date, day_D in GFI.groupby('Date', sort = True):
        print(date)
        
        trades = day_D.loc[day_D['Price'] != 0]
    
        N = 20
        f = af.trader_participation(N = N, method = 'homogenous', alpha = 2, f_min = 1, f_max = trades.shape[0], seed = 1)
        c = af.cumulative_probs(f)

        if trades.empty:
            continue

        output = af.orders(N = N, trades = trades, cumulative_probs = c)
        for n in range(N):
            
            trader_n_trades = trades.iloc[output[n], ]
           
            if trader_n_trades.empty:
                continue

            trader_n_metaorders = af.metaorders(trader_n_trades)
            
            if len(trader_n_metaorders) == 0:
                continue

            trader_n_features = af.impact_df(trader_n_metaorders, timing_method = 'immediate', impact_method = 'all')
            trader_n_features['20 AD volatility'] = GFI_AD[GFI_AD['Date'] == date]['20 AD volatility'].iloc[0] 
            
            if not trader_n_features.empty and not trader_n_features.isna().all().all():
                impact_GFI.append(trader_n_features)
                #impact_GFI = pd.concat([impact_GFI, trader_n_features], ignore_index = True)

impact_GFI   = pd.concat(impact_GFI, ignore_index = True)
exclude_cols = ['RIC', 'Date', 'Start time', 'End time']
numeric_cols = [col for col in impact_GFI.columns if col not in exclude_cols]
impact_GFI[numeric_cols] = impact_GFI[numeric_cols].apply(pd.to_numeric, errors = 'coerce')

impact_GFI['Date']          = pd.to_datetime(impact_GFI['Date']).dt.normalize()
impact_GFI['Start time']    = pd.to_datetime(impact_GFI['Start time'], format = 'mixed')
impact_GFI['End time']      = pd.to_datetime(impact_GFI['End time'], format = 'mixed')
impact_GFI['duration(min)'] = (impact_GFI['End time'] - impact_GFI['Start time']).dt.total_seconds() / 60
durations_GFI               = impact_GFI['duration(min)']

impact_GFI.to_csv('GFI_homo_20_immediate.csv', index = False)
b2.put_file('GFI_homo_20_immediate.csv', 'test_data')

2023-01-03 00:00:00
2023-01-04 00:00:00
2023-01-05 00:00:00
2023-01-06 00:00:00
2023-01-09 00:00:00
2023-01-10 00:00:00
2023-01-11 00:00:00
2023-01-12 00:00:00
2023-01-13 00:00:00
2023-01-16 00:00:00
2023-01-17 00:00:00
2023-01-18 00:00:00
2023-01-19 00:00:00
2023-01-20 00:00:00
2023-01-23 00:00:00
2023-01-24 00:00:00
2023-01-25 00:00:00
2023-01-26 00:00:00
2023-01-27 00:00:00
2023-01-30 00:00:00
2023-01-31 00:00:00
2023-02-01 00:00:00
2023-02-02 00:00:00
2023-02-03 00:00:00
2023-02-06 00:00:00
2023-02-07 00:00:00
2023-02-08 00:00:00
2023-02-09 00:00:00
2023-02-10 00:00:00
2023-02-13 00:00:00
2023-02-14 00:00:00
2023-02-15 00:00:00
2023-02-16 00:00:00
2023-02-17 00:00:00
2023-02-20 00:00:00
2023-02-21 00:00:00
2023-02-22 00:00:00
2023-02-23 00:00:00
2023-02-24 00:00:00
2023-02-27 00:00:00
2023-02-28 00:00:00
2023-03-01 00:00:00
2023-03-02 00:00:00
2023-03-03 00:00:00
2023-03-06 00:00:00
2023-03-07 00:00:00
2023-03-08 00:00:00
2023-03-09 00:00:00
2023-03-10 00:00:00
2023-03-13 00:00:00


In [ ]:
%%time
b2.get_file('top_100(Volume)/GRT.csv')
GRT = pd.read_csv('GRT.csv', parse_dates = ['DateTime', 'Date'])
GRT = GRT.rename(columns = {'Ticker' : 'RIC'})
GRT = GRT.sort_values(['DateTime', 'ExchangeSequenceNo'])

impact_GRT = pd.DataFrame(columns = ['RIC', 'Date', 'Start time', 'End time', 'daily volume', 'intraday volatility',
                                     'number child orders', 'volume traded', 'trade sign', 'impact(shortfall)',
                                     'impact(ave per trade)', 'impact(simple)', '20 AD volatility'])
impact_GRT = []
for date, day_D in GRT.groupby('Date', sort = True):
        print(date)

        trades = day_D.loc[day_D['Price'] != 0]
    
        N = 20
        f = af.trader_participation(N = N, method = 'homogenous', alpha = 2, f_min = 1, f_max = trades.shape[0], seed = 1)
        c = af.cumulative_probs(f)

        if trades.empty:
            continue

        output = af.orders(N = N, trades = trades, cumulative_probs = c)
        for n in range(N):
            
            trader_n_trades = trades.iloc[output[n], ]
           
            if trader_n_trades.empty:
                continue

            trader_n_metaorders = af.metaorders(trader_n_trades)
            
            if len(trader_n_metaorders) == 0:
                continue

            trader_n_features = af.impact_df(trader_n_metaorders, timing_method = 'immediate', impact_method = 'all')
            trader_n_features['20 AD volatility'] = GRT_AD[GRT_AD['Date'] == date]['20 AD volatility'].iloc[0]  
            
            if not trader_n_features.empty and not trader_n_features.isna().all().all():
                impact_GRT.append(trader_n_features)
                #impact_GRT = pd.concat([impact_GRT, trader_n_features], ignore_index = True)


impact_GRT   = pd.concat(impact_GRT, ignore_index = True)
exclude_cols = ['RIC', 'Date', 'Start time', 'End time']
numeric_cols = [col for col in impact_GRT.columns if col not in exclude_cols]
impact_GRT[numeric_cols] = impact_GRT[numeric_cols].apply(pd.to_numeric, errors = 'coerce')

impact_GRT['Date']          = pd.to_datetime(impact_GRT['Date']).dt.normalize()
impact_GRT['Start time']    = pd.to_datetime(impact_GRT['Start time'], format = 'mixed')
impact_GRT['End time']      = pd.to_datetime(impact_GRT['End time'], format = 'mixed')
impact_GRT['duration(min)'] = (impact_GRT['End time'] - impact_GRT['Start time']).dt.total_seconds() / 60
durations_GRT               = impact_GRT['duration(min)']

impact_GRT.to_csv('GRT_homo_20_immediate.csv', index = False)
b2.put_file('GRT_homo_20_immediate.csv', 'test_data')